# Huấn luyện RL để cân bằng Cartpole

Notebook này là một phần của [Chương trình học AI cho người mới bắt đầu](http://aka.ms/ai-beginners). Nó được lấy cảm hứng từ [hướng dẫn chính thức của PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) và [triển khai Cartpole bằng PyTorch này](https://github.com/yc930401/Actor-Critic-pytorch).

Trong ví dụ này, chúng ta sẽ sử dụng RL để huấn luyện một mô hình cân bằng một cây cột trên một xe đẩy có thể di chuyển sang trái và phải trên một trục ngang. Chúng ta sẽ sử dụng môi trường [OpenAI Gym](https://www.gymlibrary.ml/) để mô phỏng cây cột.

> **Note**: Bạn có thể chạy mã của bài học này trên máy cục bộ (ví dụ: từ Visual Studio Code), trong trường hợp đó, mô phỏng sẽ mở trong một cửa sổ mới. Khi chạy mã trực tuyến, bạn có thể cần thực hiện một số điều chỉnh mã, như được mô tả [ở đây](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Chúng ta sẽ bắt đầu bằng cách đảm bảo Gym đã được cài đặt:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Bây giờ, hãy tạo môi trường CartPole và xem cách thao tác trên nó. Một môi trường có các thuộc tính sau:

* **Action space** là tập hợp các hành động có thể thực hiện tại mỗi bước của mô phỏng  
* **Observation space** là không gian các quan sát mà chúng ta có thể thực hiện  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Hãy xem cách mô phỏng hoạt động. Vòng lặp sau sẽ chạy mô phỏng cho đến khi `env.step` không trả về cờ kết thúc `done`. Chúng ta sẽ chọn hành động một cách ngẫu nhiên bằng cách sử dụng `env.action_space.sample()`, điều này có nghĩa là thí nghiệm có thể thất bại rất nhanh (môi trường CartPole sẽ kết thúc khi tốc độ của CartPole, vị trí hoặc góc của nó vượt quá các giới hạn nhất định).

> Mô phỏng sẽ mở trong cửa sổ mới. Bạn có thể chạy mã nhiều lần và quan sát cách nó hoạt động.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Bạn có thể nhận thấy rằng các quan sát bao gồm 4 con số. Chúng là:  
- Vị trí của xe đẩy  
- Vận tốc của xe đẩy  
- Góc của cột  
- Tốc độ quay của cột  

`rew` là phần thưởng mà chúng ta nhận được ở mỗi bước. Bạn có thể thấy rằng trong môi trường CartPole, bạn được thưởng 1 điểm cho mỗi bước mô phỏng, và mục tiêu là tối đa hóa tổng phần thưởng, tức là thời gian mà CartPole có thể giữ thăng bằng mà không bị ngã.

Trong quá trình học tăng cường, mục tiêu của chúng ta là huấn luyện một **chính sách** $\pi$, mà đối với mỗi trạng thái $s$ sẽ cho chúng ta biết hành động $a$ nào cần thực hiện, về cơ bản là $a = \pi(s)$.

Nếu bạn muốn một giải pháp mang tính xác suất, bạn có thể nghĩ rằng chính sách sẽ trả về một tập hợp các xác suất cho mỗi hành động, tức là $\pi(a|s)$ sẽ biểu thị xác suất rằng chúng ta nên thực hiện hành động $a$ tại trạng thái $s$.

## Phương pháp Gradient Chính sách

Trong thuật toán RL đơn giản nhất, được gọi là **Gradient Chính sách**, chúng ta sẽ huấn luyện một mạng nơ-ron để dự đoán hành động tiếp theo.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Chúng ta sẽ huấn luyện mạng bằng cách thực hiện nhiều thí nghiệm và cập nhật mạng sau mỗi lần chạy. Hãy định nghĩa một hàm để thực hiện thí nghiệm và trả về kết quả (gọi là **trace**) - tất cả các trạng thái, hành động (và xác suất được đề xuất của chúng), và phần thưởng:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Bạn có thể chạy một tập với mạng chưa được huấn luyện và quan sát rằng tổng phần thưởng (hay còn gọi là độ dài của tập) rất thấp:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Một trong những khía cạnh khó khăn của thuật toán gradient chính sách là sử dụng **phần thưởng chiết khấu**. Ý tưởng là chúng ta tính toán vector tổng phần thưởng tại mỗi bước của trò chơi, và trong quá trình này chúng ta chiết khấu các phần thưởng ban đầu bằng một hệ số $gamma$. Chúng ta cũng chuẩn hóa vector kết quả, vì chúng ta sẽ sử dụng nó như một trọng số để ảnh hưởng đến quá trình huấn luyện của mình:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Bây giờ hãy bắt đầu quá trình huấn luyện thực sự! Chúng ta sẽ chạy 300 tập, và trong mỗi tập, chúng ta sẽ thực hiện các bước sau:

1. Chạy thí nghiệm và thu thập dấu vết.
2. Tính toán sự chênh lệch (`gradients`) giữa các hành động đã thực hiện và các xác suất dự đoán. Sự chênh lệch càng nhỏ, chúng ta càng chắc chắn rằng hành động đã thực hiện là đúng.
3. Tính toán phần thưởng chiết khấu và nhân `gradients` với phần thưởng chiết khấu - điều này đảm bảo rằng các bước có phần thưởng cao hơn sẽ có ảnh hưởng lớn hơn đến kết quả cuối cùng so với các bước có phần thưởng thấp hơn.
4. Các hành động mục tiêu kỳ vọng cho mạng nơ-ron của chúng ta sẽ được lấy một phần từ các xác suất dự đoán trong quá trình chạy, và một phần từ các `gradients` đã tính toán. Chúng ta sẽ sử dụng tham số `alpha` để xác định mức độ mà `gradients` và phần thưởng được tính đến - đây được gọi là *tốc độ học* của thuật toán tăng cường.
5. Cuối cùng, chúng ta huấn luyện mạng nơ-ron trên các trạng thái và hành động kỳ vọng, sau đó lặp lại quy trình.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Bây giờ hãy chạy tập phim với kết xuất để xem kết quả:


In [ ]:
_ = run_episode(render=True)

Hy vọng bạn có thể thấy rằng cây cột giờ đã có thể cân bằng khá tốt!

## Mô hình Actor-Critic

Mô hình Actor-Critic là một bước phát triển tiếp theo của policy gradients, trong đó chúng ta xây dựng một mạng nơ-ron để học cả chính sách và phần thưởng ước tính. Mạng này sẽ có hai đầu ra (hoặc bạn có thể xem nó như hai mạng riêng biệt):
* **Actor** sẽ đề xuất hành động cần thực hiện bằng cách cung cấp phân phối xác suất trạng thái, giống như trong mô hình policy gradient.
* **Critic** sẽ ước tính phần thưởng có thể nhận được từ những hành động đó. Nó trả về tổng phần thưởng ước tính trong tương lai tại trạng thái hiện tại.

Hãy định nghĩa một mô hình như vậy:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Chúng ta sẽ cần chỉnh sửa một chút các hàm `discounted_rewards` và `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Bây giờ chúng ta sẽ chạy vòng lặp huấn luyện chính. Chúng ta sẽ sử dụng quy trình huấn luyện mạng thủ công bằng cách tính toán các hàm mất mát phù hợp và cập nhật các tham số của mạng:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Điểm chính

Chúng ta đã tìm hiểu hai thuật toán Học tăng cường (RL) trong bản demo này: gradient chính sách đơn giản và actor-critic phức tạp hơn. Bạn có thể thấy rằng các thuật toán này hoạt động với các khái niệm trừu tượng như trạng thái, hành động và phần thưởng - do đó chúng có thể được áp dụng cho nhiều môi trường rất khác nhau.

Học tăng cường cho phép chúng ta học chiến lược tốt nhất để giải quyết vấn đề chỉ bằng cách quan sát phần thưởng cuối cùng. Việc không cần các tập dữ liệu được gắn nhãn cho phép chúng ta lặp lại các mô phỏng nhiều lần để tối ưu hóa mô hình của mình. Tuy nhiên, vẫn còn nhiều thách thức trong RL, mà bạn có thể tìm hiểu nếu quyết định tập trung nhiều hơn vào lĩnh vực thú vị này của AI.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
